# 托管推理环境，部署人脸生成模型 PGAN

### Update python packages

In [ ]:
!pip install --upgrade pip sagemaker awscli boto3 pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

### Download PGAN-CelebAHQ-512 model

We'll download [Progressive GAN](https://arxiv.org/abs/1710.10196) pre-trained model [PGAN-CelebAHQ-512](https://pytorch.org/hub/facebookresearch_pytorch-gan-zoo_pgan/) from Torch Hub, which is trained on high-quality celebrity faces "celebAHQ" dataset, then create a model artifact `model.tar.gz` and upload it to S3:

In [ ]:
%%time
import torch

use_gpu = False
model_algorithm = 'PGAN'

# this model outputs 512 x 512 pixel images
model_pretrained_name = 'celebAHQ-512' # 'celebAHQ-256'

pgan = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
                       model_algorithm, model_name=model_pretrained_name,
                       pretrained=True, useGPU=use_gpu)

# generator network
netG = pgan.netG

dim_latent = 512
print(netG.getOutputSize())

### Test generator model with 4 new random noises.

In [ ]:
import torch
from pgan.model_tools import generate_noises, show_multiple_pictures

noises = generate_noises(4, dim_latent)

with torch.no_grad():
    images = netG(noises)

show_multiple_pictures(images)
# print(netG)

### Save PGAN model to a `.pth` file, and create a TorchServe archive from it.


In [ ]:
import time

model_name = f'model-{model_algorithm}-{model_pretrained_name}-' \
                    + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

model_folder = f'./tmp/{model_algorithm}'
!mkdir -p {model_folder}

input_shape = [1, dim_latent]
# data_shape = '{"input0":[1, 512]}'

traced_model = torch.jit.trace(netG.float().eval(), torch.zeros(input_shape).float())
# scripted_model = torch.jit.script(model)
traced_model.save(f"{model_folder}/{model_name}.pth")

In [ ]:
!rm -rf ./model/*

!cp -af {model_folder}/{model_name}.pth ./model/model.pth
!cp -af pgan ./model/code

Create a compressed tar.gz file from the model.pth file, since Amazon SageMaker expects that models are in a tar.gz file. 

<code>
model.tar.gz/
|- model.pth
|- code/
  |- inference.py
  |- requirements.txt  # only for versions 1.3.1 and higher
</code>



In [ ]:
%%time
import tarfile

with tarfile.open(f"{model_folder}/{model_name}.tar.gz", 'w:gz') as f:
    f.add(f"./model/model.pth", arcname="model.pth")
    f.add(f"./model/code", arcname="code")

### Create a boto3 session and get specify a role with SageMaker access

In [ ]:
import boto3
import sagemaker
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
account_id = role.split(':')[4]

sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

### Upload the generated model archive file to Amazon S3
Uploads the model to your default Amazon SageMaker S3 bucket under the models directory

In [ ]:
s3_model_location = sess.upload_data(path=f"{model_folder}/{model_name}.tar.gz",
                              key_prefix=f"artifacts/models")

### Define a class PganPredictor, with serializer and deserializer for x-npy format

In [ ]:
from sagemaker.predictor import RealTimePredictor

def serialize(nparray, content_type='application/x-npy'):
    from io import BytesIO
    import numpy as np

    if content_type == 'application/json':
        deserialized = json.dumps(nparray.tolist())
    else:
        array_like = nparray.tolist()
        buffer = BytesIO()
        np.save(buffer, array_like)
        deserialized = buffer.getvalue()

    return deserialized


def deserialize(serialized, content_type='application/x-npy'):
    from io import BytesIO
    import numpy as np
    from botocore.response import StreamingBody

    if isinstance(serialized, StreamingBody):
        stream = BytesIO(serialized.read())
    else:
        stream = BytesIO(serialized)

    np_output = np.load(stream, allow_pickle=True)
    
    return np_output


class PganPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=serialize, 
                         deserializer=deserialize, content_type='application/x-npy',
                         accept='application/x-npy')

### Create a PyTorch model from model archive

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

sm_model = PyTorchModel(model_data=s3_model_location,
                     role=role,
                     predictor_cls=PganPredictor,
                     name=model_name,
                     entry_point='inference.py',
                     py_version="py3",
                     framework_version="1.5",
                     sagemaker_session=sess)

### Deploy model

In [ ]:
%%time
import time

endpoint_name = f'endpoint-{model_algorithm}-{model_pretrained_name}-' \
                    + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

predictor = sm_model.deploy(endpoint_name=endpoint_name,
#                          instance_type='local',
                         instance_type='ml.c5.xlarge',
                         initial_instance_count=1)
                            

### Invoke the endpoint

Let's test with a cat image.

In [ ]:
%%time
from pgan.model_tools import generate_noises, show_multiple_pictures

images = []
for i in range(0, 2):
    noises = generate_noises(1)
    output = predictor.predict(noises)
    images.append(output[0])

show_multiple_pictures(images)

In [ ]:
sm_model.delete_model()

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)

### Debug

In [ ]:
%cd ./model/code
import json
from inference import context
import handler

import importlib
importlib.reload(handler)

ctxt = context("..")

noises = generate_noises(4)
x = serialize(noises.numpy())

output = handler.handle([{'data':x}], ctxt)

show_multiple_pictures(output[0])

%cd ../..

In [ ]:
%cd ./model/code
import json
import inference

import importlib

importlib.reload(inference)

noises = generate_noises(4)
noises_serialized = serialize(noises.numpy())

model = inference.model_fn("..")

input_data = inference.input_fn(noises_serialized, 'application/x-npy')

prediction = inference.predict_fn(input_data, model)
images_serialized = inference.output_fn(prediction, 'application/x-npy')

show_multiple_pictures(deserialize(images_serialized))

%cd ../..